# Income

In [4]:
#the rule of today game is the following:
#if 8 person submit the answer in the chat, then we continue...
#so if you guys are faster, than we can finish this early...
#chaky will not CODE a single line....he will only copy you guys code....

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/income.csv')
#get this file from the google classroom

In [3]:
df.head()

,age,sex,education,education-num,marital-status,workclass,occupation,hours-per-week,income,label
0,27,Male,HS-grad,9,Never-married,Private,Craft-repair,40,<=50K,0
1,47,Male,Masters,14,Married,Local-gov,Exec-managerial,50,>50K,1
2,59,Male,HS-grad,9,Divorced,Self-emp,Prof-specialty,20,<=50K,0
3,38,Female,Prof-school,15,Never-married,Federal-gov,Prof-specialty,57,>50K,1
4,64,Female,11th,7,Widowed,Private,Farming-fishing,40,<=50K,0


In [5]:
#please check the class label - test task
df['label'].value_counts(normalize=True)
#again, deep learning should be able to handle this for us
#but you can always use imblearn library

0    0.723333
1    0.276667
Name: label, dtype: float64

## 2. Categorify

Please categorify all categorical columns

sex, education-num, marital-status, workclass, occupation

In [8]:
cat_cols  = ['sex', 'education-num', 'marital-status', 'workclass', 'occupation']
cont_cols = ['age', 'hours-per-week'] 
y         = ['label']

In [9]:
#change all cat_cols in type category
#why need to categorify?
#underhood:  things become integers, but you don't lose the string

for col in cat_cols:
    df[col] = df[col].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             30000 non-null  int64   
 1   sex             30000 non-null  category
 2   education       30000 non-null  object  
 3   education-num   30000 non-null  category
 4   marital-status  30000 non-null  category
 5   workclass       30000 non-null  category
 6   occupation      30000 non-null  category
 7   hours-per-week  30000 non-null  int64   
 8   income          30000 non-null  object  
 9   label           30000 non-null  int64   
dtypes: category(5), int64(3), object(2)
memory usage: 1.3+ MB


## 3. Create embedding sizes

In [10]:
cat_size = [len(df[col].cat.categories) for col in cat_cols]
emb_size = [(size,min(50,size//2)) for size in cat_size]
emb_size

[(2, 1), (14, 7), (6, 3), (5, 2), (12, 6)]

In [12]:
#so can you guys explain to yourself, e.g., what is (12, 6) mean?
# df['occupation'].unique()
#each of the occupation is represent by a unique vector of size 6....

## 4. Combine all categorical column into one tensor

In [ ]:
# np.stack [, 1]

# shape (all sample size, features)

# convert it into tensor; make sure it's torch.int32

## 5. Combine all continuous column into one tensor

In [ ]:
# np.stack [, 1]

# shape (all sample size, features)

#make sure it's torch.float32 for the tensor